# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 11, 12, 10

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [1]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
# pattern = ['STOCH', 'RSI', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 300
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [15, 20, 25, 30, 35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                        'slowd_period': [3], 'low_bound': [10, 15, 20, 25, 30]},
                'LinearReg': {'timeperiod': [6, 8, 10], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is sell
Number of combinations is 75


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [03:09<00:00,  2.53s/it]


# Check local statistics

In [2]:
# 15m/1h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('forecast_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
24,STOCH_RSI,12,35,9,7,3,30,87.96,57.42,63.87,75.07,79.83,80.95,79.83,77.31,80.67,83.19,84.87,84.03,87.39,86.83,86.27,86.27,84.87,84.87,82.35,78.71,73.39,69.75,69.75,70.03,-0.43,-0.21,-0.33,-1.00,-1.23,-2.19,-2.14,-1.88,-2.11,-2.31,-2.11,-2.08,-2.23,-2.36,-2.43,-2.26,-2.29,-1.98,-1.70,-1.22,-1.03,-0.89,-0.93,-0.91,357,78.978333,-1.593750,3205.265000,-568.968750
23,STOCH_RSI,12,35,9,7,3,25,87.20,56.55,62.80,75.30,80.06,81.85,79.76,78.27,81.55,83.63,86.31,84.82,88.10,87.20,86.61,86.61,85.12,85.42,82.74,78.87,74.11,70.24,70.24,70.54,-0.42,-0.20,-0.32,-0.99,-1.22,-2.35,-2.28,-1.90,-2.14,-2.32,-2.11,-2.10,-2.24,-2.36,-2.41,-2.26,-2.30,-1.98,-1.71,-1.21,-1.03,-0.90,-0.93,-0.93,336,79.329167,-1.608750,3134.600000,-540.540000
22,STOCH_RSI,12,35,9,7,3,20,86.38,56.27,60.57,76.34,80.29,83.15,81.36,79.21,83.15,86.38,88.17,85.66,89.25,88.89,88.53,88.89,87.46,87.46,84.95,79.93,73.84,70.25,70.61,71.33,-0.36,-0.16,-0.28,-0.96,-1.25,-2.53,-2.55,-2.05,-2.28,-2.44,-2.14,-2.11,-2.25,-2.36,-2.44,-2.33,-2.32,-2.06,-1.80,-1.21,-1.02,-0.89,-0.93,-0.96,279,80.346667,-1.653333,2886.720000,-461.280000
48,STOCH_RSI,14,35,9,7,3,25,84.29,54.64,63.21,76.43,80.36,82.50,81.79,81.07,83.57,84.64,87.50,86.43,88.93,87.14,86.79,85.36,84.29,84.29,81.43,76.79,72.14,68.57,68.93,68.93,-0.40,-0.15,-0.41,-1.00,-1.15,-2.75,-2.89,-2.39,-2.49,-2.62,-2.24,-2.19,-2.35,-2.37,-2.42,-2.25,-2.23,-1.91,-1.70,-1.00,-0.89,-0.81,-0.89,-0.87,280,79.167500,-1.682083,2566.900000,-470.983333
49,STOCH_RSI,14,35,9,7,3,30,85.28,56.86,63.88,75.59,79.60,80.94,81.27,79.93,82.61,83.95,85.95,85.28,87.96,86.29,85.95,84.62,83.28,83.28,80.60,76.59,71.57,67.56,67.56,67.89,-0.42,-0.22,-0.43,-1.01,-1.14,-2.61,-2.69,-2.24,-2.42,-2.59,-2.16,-2.14,-2.29,-2.36,-2.41,-2.22,-2.20,-1.90,-1.65,-1.02,-0.88,-0.80,-0.88,-0.86,299,78.512083,-1.647500,2545.112917,-492.602500
47,STOCH_RSI,14,35,9,7,3,20,83.47,53.39,61.02,78.39,80.93,84.32,84.75,83.05,86.02,87.71,89.83,87.29,90.25,89.41,88.98,88.56,87.29,86.86,83.90,77.97,72.03,69.49,69.92,70.34,-0.35,-0.11,-0.35,-1.00,-1.23,-2.84,-2.98,-2.53,-2.55,-2.64,-2.35,-2.31,-2.35,-2.37,-2.45,-2.33,-2.25,-2.00,-1.72,-0.99,-0.87,-0.81,-0.90,-0.91,236,80.632083,-1.716250,2509.171667,-405.035000
74,STOCH_RSI,16,35,9,7,3,30,84.12,56.65,64.81,76.82,81.12,84.98,86.70,84.98,87.12,86.27,87.55,85.84,89.70,88.84,87.98,87.12,85.84,84.98,81.97,76.82,71.24,66.95,68.67,69.53,-0.40,-0.21,-0.46,-1.02,-1.14,-3.02,-3.19,-2.65,-2.68,-2.81,-2.56,-2.45,-2.62,-2.43,-2.51,-2.26,-2.20,-1.90,-1.62,-0.98,-0.83,-0.77,-0.91,-0.91,233,80.275000,-1.772083,2394.075000,-412.895417
73,STOCH_RSI,16,35,9,7,3,25,83.03,54.59,64.68,77.52,81.65,86.24,87.16,85.78,88.07,87.16,89.45,87.16,90.37,89.45,88.53,87.61,86.70,85.78,82.57,76.61,71.56,67.89,70.18,70.64,-0.36,-0.12,-0.45,-1.01,-1.13,-3.23,-3.31,-2.74,-2.72,-2.82,-2.59,-2.45,-2.65,-2.46,-2.50,-2.26,-2.20,-1.92,-1.62,-0.98,-0.84,-0.78,-0.92,-0.91,21

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [12], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [30]},
                'LinearReg': {'timeperiod': [6], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            